In [ ]:
from __future__ import unicode_literals, print_function
import spacy

from spacy import displacy

import plac
import random
from pathlib import Path
from spacy.util import minibatch, compounding

LABEL = "SPORT"

TRAIN_DATA = [
    (
        "Basketball is a game that is full of scoring and defense.",
        {"entities": [(0, 10, LABEL)]},
    ),
    ("Do they start?", {"entities": []}),
    (
        "basketball is a game that is full of scoring and defense",
        {"entities": [(0, 10, LABEL)]},
    ),
    ("basketball is a cruel game that sometimes results in heartbreak", {"entities": [(0, 10, LABEL)]}),
    (
        "Lebron James and Kobe Bryant all played basketball",
        {"entities": [(42, 52, LABEL)]},
    ),
    ("basketball?", {"entities": [(0, 10, LABEL)]}),
    ("superman", {"entities": []}),
]

print(2)

@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    new_model_name=("New model name for model meta.", "option", "nm", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int),
)

def main(model=None, new_model_name="animal", output_dir=None, n_iter=30):
    
    random.seed(0)
    if model is not None:
        nlp = spacy.load(model)  
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  
        print("Created blank 'en' model")
    
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
   
    else:
        ner = nlp.get_pipe("ner")

    ner.add_label(LABEL)  
    
    ner.add_label("FRUIT")
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    move_names = list(ner.move_names)
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  
        sizes = compounding(1.0, 4.0, 1.001)
        
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            batches = minibatch(TRAIN_DATA, size = sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                print("Got in the batch annotation section")
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)

    # test the trained model
    test_text = "Do you like basketball?"
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta["name"] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        #nlp3 = spacy.load(output_dir)
        # Check the classes have loaded back consistently
        assert nlp2.get_pipe("ner").move_names == move_names
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)


if __name__ == "__main__":
    plac.call(main)
    